In [1]:
#uruchomienie sesji SPARKA
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import *
spark = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1674389236110_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#Liczba postów na przestrzeni czasu
dfPosts = spark.read.parquet('s3://stackexchangebeer1/parquet/posts.parquet')
df = dfPosts.groupBy('CreationDate').agg(f.count("CreationDate").alias("NumberOfPosts"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
%%display
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [11]:
#Porównanie najwyżej i najniżej ocenianych pytań
dfPost1 = dfPosts.select("AnswerCount","Tags","Title","Score")
dfPostNoNull = poster.dropna("any")
df = dfPostNoNull.select("Title","Tags","AnswerCount","Score").sort(f.col("Score").desc()).show(5,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------------------------+----------------------------------------------+-----------+-----+
|Title                                                            |Tags                                          |AnswerCount|Score|
+-----------------------------------------------------------------+----------------------------------------------+-----------+-----+
|Do different beer glass shapes really make a difference in taste?|glassware taste aroma                         |5          |68   |
|Do cans change the taste of beer?                                |storage cans                                  |9          |46   |
|Where can I find open APIs about beer?                           |breweries                                     |3          |45   |
|How can I tell whether a beer will improve with age?             |aging bottle-conditioning cellaring freshness |8          |43   |
|What is the difference between an ale and a lager?               |br

In [12]:
#Porównanie najwyżej i najniżej ocenianych pytań
dfPostNoNull.createOrReplaceTempView("DfPosts")
spark.sql("select Title,Tags,AnswerCount,Score from DfPosts ORDER BY Score asc").show(5,truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------+---------------------------------------------------+-----------+-----+
|Title                                           |Tags                                               |AnswerCount|Score|
+------------------------------------------------+---------------------------------------------------+-----------+-----+
|Why are breweries against freezing pint glasses?|temperature glassware                              |4          |-7   |
|What is your favorite beer & food pairing?      |pairing                                            |3          |-3   |
|Beer recommendations for a noob                 |taste recommendations breweries flavor craft-beers |3          |-3   |
|Does a Sober Scottish man really exists?        |drinking craft-beers scotch                        |1          |-3   |
|Cheapest Gin in the UK?                         |spirits gin online                                 |4          |-2   |
+-------------------------------

In [21]:
#Top N tagów które wygenerowały najwięcej wyświetleń
dfTags = spark.read.parquet('s3://stackexchangebeer1/parquet/tags.parquet')
dfTags1 = dfTags.select("TagName","Count")
dfTags1.createOrReplaceTempView("DfTags")
df = spark.sql("select TagName, Count as Views from DfTags ORDER BY Count desc limit 10")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+-----+
|        TagName|Views|
+---------------+-----+
|           wine|  137|
|          taste|  120|
|        history|   85|
|recommendations|   78|
|        brewing|   69|
|        storage|   64|
|specialty-beers|   60|
|         health|   59|
|      breweries|   56|
|        whiskey|   55|
+---------------+-----+

In [23]:
%%display
df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [14]:
dfTags.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- Count: integer (nullable = true)
 |-- PostId: integer (nullable = true)
 |-- Id: integer (nullable = true)
 |-- TagName: string (nullable = true)
 |-- WikiPostId: integer (nullable = true)

In [27]:
#najczęściej występujące słowa w tytułach

#funkcja usuwająca niepotrzebne znaki oraz zmniejszająca wszystkie litery
from pyspark.sql.functions import regexp_replace, trim, col, lower

def remove_chars(column):
    return trim(lower(regexp_replace(column, '([^\s\w_]|_)+', ''))).alias('Title')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#aplikacja funkcji do DataFrame'u składjącego się z kolumny Title
df = dfPosts.select("Title").select(remove_chars(col('Title')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
#podzielenie słów oraz ich podliczenie
from pyspark.sql import functions as f 

df.withColumn('word', f.explode(f.split(f.col('Title'), ' ')))\
    .groupBy('word')\
    .count()\
    .sort('count', ascending=False)\
    .show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-----+
| word|count|
+-----+-----+
| beer|  372|
|  the|  330|
|    a|  309|
| what|  255|
|   in|  247|
|   to|  245|
|   is|  243|
|   of|  231|
|  how|  147|
|  are|  142|
|  and|  142|
| wine|  128|
|  for|  125|
|  why|  111|
|   do|  105|
|  can|  104|
|beers|  100|
|    i|   90|
| with|   82|
|there|   82|
+-----+-----+
only showing top 20 rows